# Liver image pre processing

In this notebook, the images in DICOM format are treated for each patient and save in a more numpy usable format. 
The image can be contrast enhanced.

# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import re
import pydicom as dicom #reading dicom images 
#import dicom
import zipfile
import shutil
import cv2
from tqdm import tqdm
from skimage import exposure

# Utilities Functions

In [2]:
def get_seriesId_idx(df):
    """
    Based on the content of the columns, defines which one is the 
    'SeriesId'
    
    Returns the index of the columns containing the SeriesId
    """
    
    #finds colums where non unique values are found
    unique_cols=list()
    for idx, col in enumerate(df.columns):
        if len(df[col].unique()) > 1:
            unique_cols.append(idx)
            
    row= df.iloc[1,unique_cols] #pick the first row
    for row_idx, x in enumerate(row) :
        if not re.search('[a-zA-Z]|\-', x):
            row_idx = row_idx
            break
            
    return unique_cols[row_idx]

def get_filename_idx(df):
    """
    Based on the content of the columns, defines which one is the 
    'filename'
    
    Returns the index of the columns containing the Filename
    """
    
    #finds colums where non unique values are found
    unique_cols=list()
    for idx, col in enumerate(df.columns):
        if len(df[col].unique()) > 1:
            unique_cols.append(idx)
            
    row= df.iloc[1,unique_cols] #pick the first row
    for row_idx, x in enumerate(row) :
        if x.endswith('.dcm'):
            row_idx = row_idx
            break #stop the function
            
    return unique_cols[row_idx]

def get_series_description(df, start_string):
    """
    
    """
    
    #finds colums where non unique values are found
    unique_cols=list()
    for idx, col in enumerate(df.columns):
        if len(df[col].unique()) > 1:
            unique_cols.append(idx)
            
    row = df.iloc[1,unique_cols] #pick the first row
    row_desc = False

    for row_idx, x in enumerate(row) :
        if x.startswith(start_string):
            row_desc = row_idx
    
    if row_desc == False:
        raise NameError('Description not found')
    else:
        pass
    
    return unique_cols[row_desc]
       
def extract_mapped_img_filename(path):
    """
    Gets the name of a mapped_mri file
    """
    try: #manifest.CVS
        manifest='manifest.cvs'
        df = pd.read_csv(path+manifest, index_col=False)
    except FileNotFoundError: #manifest.CSV
        manifest='manifest.csv'
        df = pd.read_csv(path+manifest, index_col=False)
    
    #get the labels of the columns of interest
    try :
        img_type = df.columns[get_series_description(df, 'ShMO')]

        filename_lab = df.columns[get_filename_idx(df)]        
        
        #images with description : 'LIVER_T1MAP' label
        mapped_img_filename = df[df[img_type].str.endswith('T1MAP') == True][filename_lab].tolist()
        
        if len(mapped_img_filename) >2:
            raise NameError
        else:
            return mapped_img_filename[0]
    
    except NameError:
         pass

# File managing functions

In [3]:
def unzip(zip_path, zip_, contrast = False):
    """
    Extract the .csv file, extract only correct mapped-MRI image.
    """
    error_folder= []
    dst_folder = zip_path + zip_.replace('.zip', '/')
    #if file allready exists, replace
    try:
        os.mkdir(dst_folder)
    except FileExistsError: 
        shutil.rmtree(dst_folder)
        os.mkdir(dst_folder)
         
    # Create a ZipFile Object and load sample.zip in it
    with zipfile.ZipFile(zip_path + zip_ , 'r') as zipObj:
       # Get a list of all archived file names from the zip
        try:
            manifest = [f for f in zipObj.namelist() if f.startswith('manifest')][0]
            zipObj.extract(manifest, dst_folder)
            
            filename = extract_mapped_img_filename(dst_folder)

            with zipfile.ZipFile(zip_path + zip_, 'r') as zipObj2:
                zipObj2.extract(filename, dst_folder)

            ds = dicom.dcmread(os.path.join(dst_folder,filename))
            x = ds.pixel_array
            name = zip_.split('_')[0]
            
            if contrast :
                x =  exposure.equalize_adapthist(x, clip_limit=0.03)
            else:#no preporcessing
                pass 
            
            #normalization and int conversion
            image = np.uint(x * (255/np.max(np.max(x))))
            cv2.imwrite(zip_path+name+'.jpg',image)
            shutil.rmtree(dst_folder)
            return True
        
        except (IndexError, NameError, KeyError) as err:
            print(f"Error {zip_path + zip_}: {err}")
            #print('Error')
            shutil.rmtree(dst_folder)
            return False
            
def unzip_jpg(path, dst_folder, file_end_str='_2_0.zip', raw_folder= None, contrast= False):
    """
    From the downloaded from the Ukbb in the path folder,
    Extract the mri-mapped image, rename it (as the patientID) and move it to the dst_folder
    Move the zip files to raw_folder
    """
    zip_files = [z for z in os.listdir(path) if z.endswith(file_end_str)]
    nb_zip=len(zip_files)
    print('\n{} zip files.\n\n'.format(nb_zip))
    
    if raw_folder:
        os.makedirs(raw_folder+'Untreated/', exist_ok = True)
        os.makedirs(raw_folder+'Treated/', exist_ok = True)
    #moving donwloaded files
    for z in tqdm(zip_files):
        #moving zip file
        #os.rename(path+z, dst= dst_folder+z)
        shutil.copy(path+z, dst= dst_folder+z)
        
        #Unziping
        if not unzip(dst_folder, z, contrast):
            if raw_folder:
                shutil.move(dst_folder+z, raw_folder+'Untreated/'+z)
        else:#remove zip file
            if raw_folder:
                shutil.move(dst_folder+z, raw_folder+'Treated/'+z)
            os.remove(dst_folder+z) 

def contrast(input_directory, output_directory):
    """
    Applies a contrast enhancement on the images in the input_folder and stores the
    modified images in the output_folder
    """
    
    images = os.listdir(input_directory)
    
    for image in tqdm(images):
        try:
            x = cv2.imread(input_directory + image)
            x = exposure.equalize_adapthist(x, clip_limit=0.03)
            x = np.uint(x * (255/np.max(np.max(x))))
            cv2.imwrite(output_directory + image, x)
        except AttributeError:
            print(image)

# Run scripts

In [4]:
source = '/scratch1/06568/joyce_w/field_20204/'
destination = '/scratch1/06568/joyce_w/field_20204_2_unzip/'

unzip_jpg(source, destination, file_end_str='_2_0.zip', raw_folder= None, contrast= False)


69103 zip files.




  0%|          | 5/69103 [00:01<4:05:38,  4.69it/s] /tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
  0%|          | 25/69103 [00:03<1:36:07, 11.98it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
  0%|          | 29/69103 [00:03<1:44:16, 11.04it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
  0%|          | 51/69103 [00:05<1:28:36, 12.99it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with inde

Error /scratch1/06568/joyce_w/field_20204_2_unzip/4116261_20204_2_0.zip: 'There is no item named None in the archive'


  1%|          | 451/69103 [00:35<1:17:40, 14.73it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
  1%|          | 469/69103 [00:36<1:44:53, 10.91it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
  1%|          | 487/69103 [00:38<1:17:18, 14.79it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
  1%|          | 511/69103 [00:40<1:34:19, 12.12it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with 

Error /scratch1/06568/joyce_w/field_20204_2_unzip/3801082_20204_2_0.zip: 'There is no item named None in the archive'


...

 99%|█████████▉| 68285/69103 [1:53:32<00:57, 14.18it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
 99%|█████████▉| 68307/69103 [1:53:34<00:48, 16.37it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
 99%|█████████▉| 68311/69103 [1:53:34<00:47, 16.58it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
 99%|█████████▉| 68315/69103 [1:53:34<00:47, 16.61it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of da

Error /scratch1/06568/joyce_w/field_20204_2_unzip/2457748_20204_2_0.zip: 'There is no item named None in the archive'


100%|█████████▉| 68776/69103 [1:54:09<00:20, 15.67it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
100%|█████████▉| 68782/69103 [1:54:09<00:20, 15.69it/s]

Error /scratch1/06568/joyce_w/field_20204_2_unzip/3179931_20204_2_0.zip: 'There is no item named None in the archive'


/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
100%|█████████▉| 68786/69103 [1:54:09<00:24, 13.05it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
100%|█████████▉| 68792/69103 [1:54:10<00:21, 14.31it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manifest, index_col=False)
100%|█████████▉| 68799/69103 [1:54:10<00:25, 11.96it/s]/tmp/ipykernel_240446/3410310388.py:79: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(path+manife

In [4]:
input_directory = destination
output_directory = '/work2/06568/joyce_w/frontera/Abdominal_imaging/data/processed/field_20204_2_processed/'

# form jpg file to constrasted version
contrast(input_directory, output_directory)

  1%|          | 438/69103 [00:43<1:54:04, 10.03it/s]

4116261_20204_2_0.zip


  1%|          | 846/69103 [01:22<1:30:50, 12.52it/s]

3801082_20204_2_0.zip


  3%|▎         | 1768/69103 [02:51<2:12:55,  8.44it/s]

...

 99%|█████████▉| 68757/69103 [1:53:15<00:28, 12.24it/s]

2457748_20204_2_0.zip


100%|█████████▉| 68775/69103 [1:53:17<00:26, 12.43it/s]

3179931_20204_2_0.zip


100%|██████████| 69103/69103 [1:53:47<00:00, 10.12it/s]


In [7]:
source = '/scratch1/06568/joyce_w/field_20204/'
destination = '/scratch1/06568/joyce_w/field_20204_3_unzip/'

unzip_jpg(source, destination, file_end_str='_3_0.zip', raw_folder= None, contrast= False)


5134 zip files.




  7%|▋         | 348/5134 [00:06<01:28, 54.23it/s]

Error /scratch1/06568/joyce_w/field_20204_3_unzip/3635108_20204_3_0.zip: 'There is no item named None in the archive'


  8%|▊         | 390/5134 [00:07<01:32, 51.08it/s]

Error /scratch1/06568/joyce_w/field_20204_3_unzip/3222989_20204_3_0.zip: 'There is no item named None in the archive'


...

 69%|██████▉   | 3538/5134 [01:42<01:45, 15.10it/s]

Error /scratch1/06568/joyce_w/field_20204_3_unzip/2540561_20204_3_0.zip: 'There is no item named None in the archive'


 86%|████████▌ | 4398/5134 [02:36<00:43, 16.82it/s]

Error /scratch1/06568/joyce_w/field_20204_3_unzip/4601275_20204_3_0.zip: 'There is no item named None in the archive'


100%|██████████| 5134/5134 [03:23<00:00, 25.21it/s]


In [8]:
input_directory = destination
output_directory = '/work2/06568/joyce_w/frontera/Abdominal_imaging/data/processed/field_20204_3_processed/'

# form jpg file to constrasted version
contrast(input_directory, output_directory)

  7%|▋         | 371/5134 [00:29<07:09, 11.08it/s]

3635108_20204_3_0.zip


  8%|▊         | 420/5134 [00:32<04:51, 16.15it/s]

3222989_20204_3_0.zip


 28%|██▊       | 1428/5134 [01:54<04:02, 15.27it/s]

...

 71%|███████   | 3625/5134 [04:53<01:39, 15.16it/s]

2540561_20204_3_0.zip


 87%|████████▋ | 4468/5134 [06:02<00:46, 14.25it/s]

4601275_20204_3_0.zip


100%|██████████| 5134/5134 [06:56<00:00, 12.31it/s]
